In [5]:
# Autoreload
%load_ext autoreload
%autoreload 2

from src.dataset import read_bidmc_dataset, split_dataset_by_age, split_dataset_by_gender, split_datasets_to_windows
from src.metrics import metrics_table
from src.predictors import get_detector
from src.pipeline import Pipeline

dataset = read_bidmc_dataset()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
# We want a standardization followed by a butterworth filter for hr estimation
from src.filters import get_default_filter_for_metric

dataset_splits = ["gender", "age"]
smoothing_filter_names = ["moving_average", "median", None]
bandpass_filter_names = ["bessel", "butterworth", "chebyshev", "chebyshev2", "elliptic"]
detector_names = ["peak", "zcr", "mcr", "spectral"]
rate_names = ["hr", "rr"]
metrics = ["mae", "rmse", "mape"]

standardization_filter = get_default_filter_for_metric("standardize", "hr")

for dataset_split in dataset_splits:
    splits, split_names = split_dataset_by_age(dataset) if dataset_split == "age" else split_dataset_by_gender(dataset)
    print(len(splits[0]))
    windowed_datasets = split_datasets_to_windows(splits, window_size=30, window_stride=30)
    
    for rate_name in rate_names:
        f = open(f"{dataset_split}_{rate_name}.tsv", "w+")
        f.write("Bandpass\tDetector\tSmoothing\tSplit Name\tMAE\tRMSE\tMAPE\n")
        
        for detector_name in detector_names:
            for smoothing_filter_name in smoothing_filter_names:
                for bandpass_filter_name in bandpass_filter_names:
                    bandpass_filter = get_default_filter_for_metric(bandpass_filter_name, rate_name)
                    if smoothing_filter_name is None:
                        filters = [standardization_filter, bandpass_filter]
                    else:
                        smoothing_filter = get_default_filter_for_metric(smoothing_filter_name, rate_name)
                        filters = [standardization_filter, smoothing_filter, bandpass_filter]
            
                    predictor = get_detector(detector_name, rate_name)
                    pipeline = Pipeline(filters, predictor)
    
                    for windowed_dataset, split_name in zip(windowed_datasets, split_names):
                        gts = []
                        preds = []
                        for window in windowed_dataset:
                            gt = window[rate_name]
                            pred = pipeline(window['ppg'])
                            gts.append(gt)
                            preds.append(pred)
    
                        table, scores = metrics_table(gts, preds, return_scores=True)
                        f.write(f"{bandpass_filter_name}\t{detector_name}\t{smoothing_filter_name}\t{split_name}\t{scores['MAE']}\t{scores['RMSE']}\t{scores['MAPE']}\n")

        f.close()

20
12
